<a href="https://colab.research.google.com/github/marvin-math/active_inference_multiarmed_bandit/blob/main/thesis_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy
from statistics import mean
import random
import pandas as pd

#from scipy.special import psi


class BayesianBanditAgent:
    def __init__(self, num_arms, prior_mean, prior_precision, prior_alpha, prior_beta, eta, decay, ucb_weight):
        self.num_arms = num_arms
        self.prior_mean = prior_mean
        self.prior_precision = prior_precision
        self.prior_alpha = prior_alpha
        self.prior_beta = prior_beta
        self.draws = np.zeros(num_arms)
        self.posterior_dist_mean = np.ones(num_arms) # this has to be reworked
        self.posterior_variance = np.ones(num_arms)
        self.posterior_mean = np.ones(num_arms)
        self.posterior_precision = np.ones(num_arms)
        self.posterior_alpha = np.ones(num_arms)
        self.posterior_beta = np.ones(num_arms)
        self.rewards = np.zeros(num_arms)
        self.observations = np.empty(num_arms, dtype=object)
        self.observations[:] = [[] for _ in range(num_arms)]
        self.obs_mean = np.zeros(num_arms)
        self.t_dof = np.zeros(num_arms)
        self.t_mean = np.zeros(num_arms)
        self.t_precision = np.zeros(num_arms)
        self.t_sd = np.zeros(num_arms)
        self.predictive_pdf = np.ones(num_arms)
        self.entropy_predictive_pdf = np.ones(num_arms)
        self.eta = eta
        self.kl_div = np.ones(num_arms)
        self.ambiguity_term = np.ones(num_arms)
        self.expected_free_energy = np.ones(num_arms)
        self.expected_free_energies = np.empty(num_arms, dtype=object)
        self.expected_free_energies[:] = [[] for _ in range(num_arms)]
        self.decay = decay
        self.weight_current = np.ones(num_arms)
        self.moving_avg_previous = np.zeros(num_arms)
        self.weight_previous = np.zeros(num_arms)
        self.moving_avg_current = np.zeros(num_arms)
        self.thompson_mean = np.ones(num_arms)
        self.ucb_weight = ucb_weight
        self.ucb = np.ones(num_arms)
        self.action_values = np.ones(num_arms)
        self.current_reward = np.ones(num_arms)
        self.softmax_probs = np.zeros(num_arms)



    def select_lowest_expected_free_energy_arm(self):
        min_expected_free_energy = float('inf')
        best_arm = 0

        for arm in range(self.num_arms):
            self.expected_free_energy[arm] = self.calc_expected_free_energy(arm)
            self.expected_free_energies[arm].append(self.expected_free_energy[arm])
            #if self.expected_free_energy[arm] <= min_expected_free_energy: # a bit ambiguity here, but probably not relevant
                #min_expected_free_energy = self.expected_free_energy[arm]
                #best_arm = arm
        # Apply softmax function to the expected free energy values
        softmax_probs = self.softmax(self.expected_free_energy)
        self.softmax_probs = softmax_probs
        #print(self.softmax_probs)


        # Choose an arm probabilistically based on the softmax probabilities
        best_arm = np.random.choice(range(self.num_arms), p=softmax_probs)

        self.draws[best_arm] += 1

        return best_arm

    def random_agent(self):
      choice_prob1 = round(random.random(), 3)
      choice_prob2 = round(random.random(), 3)
      self.softmax_probs[0] = choice_prob1 / (choice_prob1 + choice_prob2)
      self.softmax_probs[1] = choice_prob2 / (choice_prob1 + choice_prob2)
      best_arm = np.random.choice(range(self.num_arms), p=self.softmax_probs)
      return best_arm

    def softmax(self, vector):
        # Calculate the softmax values
        exp_vals = np.exp(vector)
        probs = exp_vals / np.sum(exp_vals)
        probs = 1-probs # because we are minimizing
        return probs

    def thompson_sampling(self):
        max_thompson = -float('inf')
        best_arm = 0
        # Draw samples from the posterior distributions
        for arm in range(self.num_arms):
            self.thompson_mean[arm] = np.random.normal(self.posterior_mean[arm], (self.posterior_variance[arm]/np.sqrt(self.posterior_precision[arm])))
            if self.thompson_mean[arm] >= max_thompson:
                max_thompson = self.thompson_mean[arm]
                best_arm = arm
        softmax_probs = self.softmax(self.thompson_mean)
        self.softmax_probs = softmax_probs


        self.draws[best_arm] += 1

        return best_arm


    def ucb_action(self):
        max_ucb = -float('inf')
        best_arm = 0
        for arm in range(self.num_arms):
            # Calculate the uncertainty bonus (UCB) for each arm
            self.ucb[arm] = self.ucb_weight * np.sqrt(1 / self.posterior_precision[arm])

            # Calculate the action values (Q_t(k) + U_t(k)) for each arm
            self.action_values[arm] = self.posterior_mean[arm] + self.ucb[arm]
            #if self.action_values[arm] >= max_ucb:
                # Select the arm with the highest action value
                #max_ucb = self.action_values[arm]
                #best_arm = arm
        softmax_probs = self.softmax(self.action_values)
        self.softmax_probs = softmax_probs
        best_arm = np.random.choice(range(self.num_arms), p=softmax_probs)

        self.draws[best_arm] += 1

        return best_arm

    def update(self, arm, reward):
        # Update the posterior parameters for the selected arm
        self.rewards[arm] += reward
        self.current_reward[arm] = reward
        self.observations[arm].append(reward)
        self.obs_mean[arm] = np.mean(self.observations[arm])

        self.posterior_precision[arm] = self.draws[arm] + self.prior_precision[arm]
        self.posterior_mean[arm] = (self.prior_precision[arm] * self.prior_mean[arm] + self.draws[arm] * self.obs_mean[arm]) / self.posterior_precision[arm]
        self.posterior_alpha[arm] = self.prior_alpha[arm] + 0.5 * self.draws[arm]
        self.posterior_beta[arm] = self.prior_beta[arm] + (0.5 * np.sum((self.observations[arm]-self.obs_mean[arm])**2)) + \
                                    (self.prior_precision[arm] * self.draws[arm]*(self.obs_mean[arm]-self.prior_mean[arm])**2) / (2*(self.posterior_precision[arm]+self.draws[arm]))
        self.posterior_variance[arm] = np.sqrt(np.mean(1/np.random.gamma(self.posterior_alpha[arm], 1/self.posterior_beta[arm], size = 1000))) # this is actually sd
        self.posterior_dist_mean[arm] = mean(np.random.normal(self.posterior_mean[arm], (self.posterior_variance[arm]/np.sqrt(self.posterior_precision[arm])), size= 1000))

        # update the parameters for the next iteration
        self.prior_precision[arm] = self.posterior_precision[arm]
        self.prior_mean[arm] = self.posterior_mean[arm]
        self.prior_alpha[arm] = self.posterior_alpha[arm]
        self.prior_beta[arm] = self.posterior_beta[arm]


    def update_dynamic(self, arm, reward):
        # Update the posterior parameters for the selected arm
        self.rewards[arm] += reward
        self.current_reward[arm] = reward
        self.observations[arm].append(reward)
        self.obs_mean[arm] = np.mean(self.observations[arm])

        # weighting factor calculation
        if len(self.observations[arm]) > 0:
          self.weight_current[arm] = self.decay * self.weight_previous[arm] + 1
        else:
          self.weight_current[arm] = 1
        self.moving_avg_current[arm] = (1 - 1/self.weight_current[arm]) * self.moving_avg_previous[arm] + (1/self.weight_current[arm]) * reward

        # bayesian parameter estimation
        self.posterior_precision[arm] = self.draws[arm] + self.prior_precision[arm]
        self.posterior_mean[arm] = (self.prior_precision[arm] * self.prior_mean[arm] + self.draws[arm] * self.moving_avg_current[arm]) / self.posterior_precision[arm]
        self.posterior_alpha[arm] = self.prior_alpha[arm] + 0.5 * self.draws[arm]
        self.posterior_beta[arm] = self.prior_beta[arm] + (0.5 * np.sum((self.observations[arm]-self.moving_avg_current[arm])**2)) + \
                                    (self.prior_precision[arm] * self.draws[arm]*(self.moving_avg_current[arm]-self.prior_mean[arm])**2) / (2*(self.posterior_precision[arm]+self.draws[arm]))
        self.posterior_variance[arm] = np.sqrt(np.mean(1/np.random.gamma(self.posterior_alpha[arm], 1/self.posterior_beta[arm], size = 1000))) # this is actually sd
        self.posterior_dist_mean[arm] = mean(np.random.normal(self.posterior_mean[arm], (self.posterior_variance[arm]/np.sqrt(self.posterior_precision[arm])), size = 1000))

        # update the parameters for the next iteration
        self.prior_precision[arm] = self.posterior_precision[arm]
        self.prior_mean[arm] = self.posterior_mean[arm]
        self.prior_alpha[arm] = self.posterior_alpha[arm]
        self.prior_beta[arm] = self.posterior_beta[arm]

        # update weighting factor parameters for next iteration
        self.weight_previous[arm] = self.weight_current[arm]
        self.moving_avg_previous[arm] = self.moving_avg_current[arm]


    def posterior_predictive(self, arm):
        # Calculate the posterior predictive t-distribution parameters for the selected arm
        self.t_dof[arm] = 2*(self.posterior_alpha[arm] + 1)
        self.t_mean[arm] = self.posterior_mean[arm]
        self.t_precision[arm] = (self.posterior_beta[arm] * (1 + self.posterior_precision[arm])) / (self.posterior_precision[arm]* (1 + self.posterior_alpha[arm]))
        self.t_sd[arm] = 1/np.sqrt(self.t_precision[arm]) # scipy uses standard deviation, so we have to transform


        v = self.t_dof[arm]
        v1 = (1+v)/2
        v2 = v/2
        self.entropy_predictive_pdf[arm] = v1*(scipy.special.digamma(v1)-scipy.special.digamma(v2))+np.log(np.sqrt(v)*scipy.special.beta(v2,0.5))

        return self.entropy_predictive_pdf[arm]

    def kl_divergence(self, arm, eta):
        entropy_of_predictive = self.posterior_predictive(arm)
        self.kl_div[arm] = - eta*self.posterior_mean[arm] - entropy_of_predictive
        return self.kl_div[arm]

    def ambiguity(self, arm):
        self.ambiguity_term[arm] = 0.5 * np.log(2 * np.pi * np.e) + np.log(self.posterior_beta[arm]) + scipy.special.digamma(self.posterior_alpha[arm])
        return self.ambiguity_term[arm]

    def calc_expected_free_energy(self, arm):
        self.expected_free_energy_term = self.kl_divergence(arm, eta = self.eta) + self.ambiguity(arm)
        return self.expected_free_energy_term

    def optimal_strategy(self, true_arm_means):
        return np.argmax(true_arm_means)









In [2]:
# import the data
url = 'https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/musexp1.csv'
df1 = pd.read_csv(url)
np_array1_exp1 = df1['mu1'].to_numpy()
np_array2_exp1 = df1['mu2'].to_numpy()

url2 = 'https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/exp2data.csv'
df2 = pd.read_csv(url2)
np_array1_exp2 = df2['mu1'].to_numpy()
np_array2_exp2 = df2['mu2'].to_numpy()
np_arraychoice_exp2 = df2['choice'].to_numpy()
np_arrayreward_exp2 = df2['reward'].to_numpy()
np_arraysubject_exp2 = df2['subject'].to_numpy()

url3 ='https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/optimization_ai.csv'
df3 = pd.read_csv(url3)
np_eta = df3['eta'].to_numpy()
np_decay = df3['decay'].to_numpy()

url4 ='https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/optimization_ucb.csv'
df4 = pd.read_csv(url4)
np_ucb = df4['ucb'].to_numpy()
#np_decay = df4['decay'].to_numpy()

url5 ='https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/df_thompson.csv'
df5 = pd.read_csv(url5)
#np_decay = df5['decay'].to_numpy()

url6 ='https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/df_optimization_random.csv'
df6 = pd.read_csv(url6)
#np_decay = df6['decay'].to_numpy()

In [3]:
from pickle import TRUE
# Define the number of arms and prior hyperparameters

num_arms = 2

num_total_iterations = len(np_array1_exp2)

trials = np.zeros(num_total_iterations)
expected_free_energy_arm0 = np.zeros(num_total_iterations)
expected_free_energy_arm1 = np.zeros(num_total_iterations)
posterior_dist_mean_arm0 = np.zeros(num_total_iterations)
posterior_dist_mean_arm1 = np.zeros(num_total_iterations)
posterior_variance_arm0 = np.zeros(num_total_iterations)
posterior_variance_arm1 = np.zeros(num_total_iterations)
posterior_mean_arm0 = np.zeros(num_total_iterations)
posterior_mean_arm1 = np.zeros(num_total_iterations)
posterior_precision_arm0 = np.zeros(num_total_iterations)
posterior_precision_arm1 = np.zeros(num_total_iterations)
posterior_alpha_arm0 = np.zeros(num_total_iterations)
posterior_alpha_arm1 = np.zeros(num_total_iterations)
posterior_beta_arm0 = np.zeros(num_total_iterations)
posterior_beta_arm1 = np.zeros(num_total_iterations)
rewards = np.zeros(num_total_iterations)
cumulative_rewards = np.zeros(num_total_iterations)
cumulative_optimal_rewards = np.zeros(num_total_iterations)
thompson_mean_arm0 = np.zeros(num_total_iterations)
thompson_mean_arm1 = np.zeros(num_total_iterations)
ucb_arm0 = np.zeros(num_total_iterations)
ucb_arm1 = np.zeros(num_total_iterations)
action_values_arm0 = np.zeros(num_total_iterations)
action_values_arm1 = np.zeros(num_total_iterations)
chosen_arms = np.zeros(num_total_iterations)
optimal_arms = np.zeros(num_total_iterations)
optimal_rewards = np.zeros(num_total_iterations)
regrets = np.zeros(num_total_iterations)
cumulative_regrets = np.zeros(num_total_iterations)
etas_list = np.zeros(num_total_iterations)
decays_list = np.zeros(num_total_iterations)
true_dist_mean0 = np.zeros(num_total_iterations)
true_dist_mean1 = np.zeros(num_total_iterations)
draws0 = np.zeros(num_total_iterations)
draws1 = np.zeros(num_total_iterations)
ucb_list = np.zeros(num_total_iterations)
ambiguity_arm0 = np.zeros(num_total_iterations)
ambiguity_arm1 = np.zeros(num_total_iterations)








num_simulations = 1
num_iterations_per_simulation = len(np_array1_exp2)


## initialise the numpy arrays to be appended to dataframe
# Create empty NumPy arrays to store the results for each variable



true_arm_means_dynamic = np.empty(num_arms, dtype=object)
true_arm_means_dynamic[:] = [[] for _ in range(num_arms)]





#setup experiment environment
environment = 'dynamic'
experiment1 = False


#eta = 0.3 # exploration exploitation parameter - this parameter gives the weight the posterior mean has in the decision (higher weight -> lower kl divergence -> lower expected free energy)
#decay = 0.1 # how much are previous samples taken into account? Between 0 and 1 - higher values give more weight to older data, lower values more weight to current data
ucb_weight = 0.6

#create empty dict to store the results
result_df = pd.DataFrame(columns=['trial', 'expected_free_energy', 'posterior_dist_mean', 'posterior_variance', 'posterior_mean', 'posterior_precision', 'posterior_alpha', 'posterior_beta', 'reward', 'cumulative_reward', 'cumulative_optimal_reward', 'thompson_mean', 'ucb', 'action_values', 'chosen_arm', 'optimal_arm', 'optimal_reward', 'regret', 'cumulative_regret', 'true_dist_mean', 'draw_sd', 'decay', 'ucb_weight', 'eta', 'draws0', 'draws1', 'ambiguity_arm0', 'ambiguity_arm1'])
index = 0
# Reset cumulative variables for each simulation
cumulative_reward = 0
cumulative_regret = 0
cumulative_optimal_reward = 0

for participant in range(44):
    mean_of_true_rewards = 5
    mean_of_true_sd = 3
    prior_mean = np.ones(num_arms)
    prior_precision = np.ones(num_arms)
    prior_alpha = np.ones(num_arms)
    prior_beta = np.ones(num_arms)*2

    mu1 = np_array1_exp2[np_arraysubject_exp2 == (participant+1)]
    mu2 = np_array2_exp2[np_arraysubject_exp2 == (participant+1)]
    print(participant)
    eta = np_eta[participant]
    decay = np_decay[participant]
    #ucb_weight = np_ucb[participant]
    #eta = 0.3 # dummy





    # Create the Bayesian bandit agent
    agent = BayesianBanditAgent(num_arms, prior_mean, prior_precision, prior_alpha, prior_beta, eta, decay, ucb_weight)

    # Simulate the multi-armed bandit problem for a certain number of iterations
    if environment == 'stationary':
      num_iterations = 5000
      arm_means =  [3, 10]      #np.random.normal(mean_of_true_rewards, 1.0, num_arms)  # True mean rewards for each arm
      arm_sds = [10, 2]      #np.random.gamma(mean_of_true_sd, 2.0, num_arms) # true sd for each arm


    elif environment == 'dynamic':
      if experiment1 == True:
        true_arm_means_dynamic[0] = np_array1_exp1
        true_arm_means_dynamic[1] = np_array2_exp1

      elif experiment1 == False:
        true_arm_means_dynamic[0] = np_array1_exp2
        true_arm_means_dynamic[1] = np_array2_exp2
      num_iterations = len(true_arm_means_dynamic[0])



    for t in range(len(mu1)):
      # Select an arm
      chosen_arm = agent.select_lowest_expected_free_energy_arm()

      if environment == 'stationary':

        # Simulate the reward for the chosen arm (normally distributed)
        reward = np.random.normal(arm_means[chosen_arm], arm_sds[chosen_arm])
        cumulative_reward += reward

        # Update the agent with the observed reward
        agent.update_dynamic(chosen_arm, reward)
      elif environment == 'dynamic':
        true_sd = np.sqrt(10)
        if experiment1 == True:
          if chosen_arm == 0:
            reward = np.random.normal(mu1[t], true_sd)
          elif chosen_arm == 1:
            reward = 0
        elif experiment1 == False:
          if chosen_arm == 0:
            reward = np.random.normal(mu1[t], true_sd)
          elif chosen_arm == 1:
            reward = np.random.normal(mu2[t], true_sd)


        cumulative_reward += reward
        trial_number = t+1

        agent.update_dynamic(chosen_arm, reward)
        # Append values to the numpy array after each trial
        true_arm_means = [mu1[t], mu2[t]]
        optimal_arm = agent.optimal_strategy(true_arm_means)
        optimal_reward = np.random.normal(true_arm_means[optimal_arm], true_sd)
        cumulative_optimal_reward += optimal_reward
        regret = optimal_reward - reward
        cumulative_regret += regret

        trials[index] = t + 1

        # Update the values for both arms after each trial
        chosen_arms[index] = chosen_arm
        optimal_arms[index] = optimal_arm
        optimal_rewards[index] = optimal_reward
        regrets[index] = regret
        cumulative_regrets[index] = cumulative_regret
        etas_list[index] = eta
        decays_list[index] = decay
        ucb_list[index] = ucb_weight
        expected_free_energy_arm0[index] = agent.calc_expected_free_energy(0)
        expected_free_energy_arm1[index] = agent.calc_expected_free_energy(1)
        posterior_dist_mean_arm0[index] = agent.posterior_dist_mean[0]
        posterior_dist_mean_arm1[index] = agent.posterior_dist_mean[1]
        posterior_variance_arm0[index] = agent.posterior_variance[0]
        posterior_variance_arm1[index] = agent.posterior_variance[1]
        posterior_mean_arm0[index] = agent.posterior_mean[0]
        posterior_mean_arm1[index] = agent.posterior_mean[1]
        posterior_precision_arm0[index] = agent.posterior_precision[0]
        posterior_precision_arm1[index] = agent.posterior_precision[1]
        posterior_alpha_arm0[index] = agent.posterior_alpha[0]
        posterior_alpha_arm1[index] = agent.posterior_alpha[1]
        posterior_beta_arm0[index] = agent.posterior_beta[0]
        posterior_beta_arm1[index] = agent.posterior_beta[1]
        rewards[index] = reward
        cumulative_rewards[index] = cumulative_reward
        cumulative_optimal_rewards[index] = cumulative_optimal_reward
        thompson_mean_arm0[index] = agent.thompson_mean[0]
        thompson_mean_arm1[index] = agent.thompson_mean[1]
        ucb_arm0[index] = agent.ucb[0]
        ucb_arm1[index] = agent.ucb[1]
        action_values_arm0[index] = agent.action_values[0]
        action_values_arm1[index] = agent.action_values[1]
        true_dist_mean0[index] = true_arm_means[0]
        true_dist_mean1[index] = true_arm_means[1]
        draws0[index] = agent.draws[0]
        draws1[index] = agent.draws[1]
        ambiguity_arm0[index] = agent.ambiguity_term[0]
        ambiguity_arm1[index] = agent.ambiguity_term[1]




        index += 1

# Create a pandas DataFrame with the collected data
result_df = pd.DataFrame({
    'trial': trials,
    'eta': etas_list,
    'decay': decays_list,
    'ucb': ucb_list,
    'expected_free_energy_arm0': expected_free_energy_arm0,
    'expected_free_energy_arm1': expected_free_energy_arm1,
    'posterior_dist_mean_arm0': posterior_dist_mean_arm0,
    'posterior_dist_mean_arm1': posterior_dist_mean_arm1,
    'posterior_variance_arm0': posterior_variance_arm0,
    'posterior_variance_arm1': posterior_variance_arm1,
    'posterior_mean_arm0': posterior_mean_arm0,
    'posterior_mean_arm1': posterior_mean_arm1,
    'posterior_precision_arm0': posterior_precision_arm0,
    'posterior_precision_arm1': posterior_precision_arm1,
    'posterior_alpha_arm0': posterior_alpha_arm0,
    'posterior_alpha_arm1': posterior_alpha_arm1,
    'posterior_beta_arm0': posterior_beta_arm0,
    'posterior_beta_arm1': posterior_beta_arm1,
    'rewards': rewards,
    'cumulative_rewards': cumulative_rewards,
    'cumulative_optimal_rewards': cumulative_optimal_rewards,
    'thompson_mean_arm0': thompson_mean_arm0,
    'thompson_mean_arm1': thompson_mean_arm1,
    'ucb_arm0': ucb_arm0,
    'ucb_arm1': ucb_arm1,
    'action_values_arm0': action_values_arm0,
    'action_values_arm1': action_values_arm1,
    'chosen_arm': chosen_arms,
    'optimal_arm': optimal_arms,
    'optimal_reward': optimal_rewards,
    'regret': regrets,
    'cumulative_regret': cumulative_regrets,
    'true_dist_mean0': true_dist_mean0,
    'true_dist_mean1': true_dist_mean1,
    'draw_sd': np.sqrt(10),
    'draws0': draws0,
    'draws1': draws1,
    'ambiguity_arm0': ambiguity_arm0,
    'ambiguity_arm1': ambiguity_arm1
})









result_df.to_csv('/content/simulation_results_ai.csv', index=False)



#print("Posterior Predictive PDF for New Data Point:", posterior_predictive_pdf)
print(f"the posterior means are: {np.mean(agent.posterior_dist_mean)}")
print(f"the posterior sd is: {np.mean(agent.posterior_variance)}")
#print(f"the true arm means are: {np.mean(true_arm_means_dynamic[0])} and {np.mean(true_arm_means_dynamic[1])}")
print(f"the true arm sds are: {np.sqrt(10)}")
print(f"the cumulative reward is: {cumulative_reward}")

#print(f"entropy of t-distribution is: {entropy}")
#print(f"kl_divergence is: {agent.kl_divergence}")
#print(f"ambiguity is: {agent.ambiguity}")
print(f"the expected free energy of arm 1: {np.mean(expected_free_energy_arm0)} and arm 2: {np.mean(expected_free_energy_arm1)}")
#print(f"draws: {agent.draws}")
#print(f"kl divergences: {np.mean(agent.kl_div)}")
print(f"total draws: {agent.draws[0] + agent.draws[1]}")
print(f"the entropies of the predictives are: {np.mean(agent.entropy_predictive_pdf[0])} and {np.mean(agent.entropy_predictive_pdf[1])}")
print(f"the kl divergences are: {np.mean(agent.kl_div[0])} and {np.mean(agent.kl_div[1])}")
print(f"the ambiguity terms are: {np.mean(agent.ambiguity_term[0])} and {np.mean(agent.ambiguity_term[1])}")


for arm in range(num_arms):
  #print(f"mean free energy per arm: {np.mean(agent.expected_free_energies[arm])}")
  print(f"arm {arm} has been chosen {int(agent.draws[arm])} times")
  #print(f"observation matrix: {agent.observations}")









0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
the posterior means are: 0.5637995770763581
the posterior sd is: 11.783349847536176
the true arm sds are: 3.1622776601683795
the cumulative reward is: 8896.200716152105
the expected free energy of arm 1: 15.856971879105464 and arm 2: 15.877400725269474
total draws: 200.0
the entropies of the predictives are: 1.4191022328179397 and 1.4191825100249078
the kl divergences are: -1.3828097976823162 and -1.6754255787612635
the ambiguity terms are: 22.006456699253338 and 21.92926147944851
arm 0 has been chosen 110 times
arm 1 has been chosen 90 times


In [9]:
from scipy.optimize import Bounds, minimize

results_df = pd.DataFrame(columns=['Participant', 'eta', 'decay', 'Optimal Likelihood'])

# initialise dataframe to save the values


num_arms = 2
num_iterations = len(np_arrayreward_exp2)
p = np.empty(num_arms, dtype=object)
p[:] = [[] for _ in range(num_arms)]

# Define the objective function with proper argument structure
def neg_fe_log_lik(parameters, *args):
    ucb_weight = 0.6
    eta = parameters[0]
    decay = parameters[1]
    prior_mean = np.ones(num_arms) * 0
    prior_precision = np.ones(num_arms)
    prior_alpha = np.ones(num_arms)
    prior_beta = np.ones(num_arms) * 2
    negloglikelihood = 0

    agent = BayesianBanditAgent(num_arms, prior_mean, prior_precision, prior_alpha, prior_beta, eta, decay, ucb_weight)



    for t in range(len(participant_choices)):
        #best_arm = agent.select_lowest_expected_free_energy_arm()
        #best_arm = agent.thompson_sampling()
        best_arm = agent.select_lowest_expected_free_energy_arm()
        if participant_choices[t] == 0:
            negloglikelihood += np.log(agent.softmax_probs[0]) # the posterior variance is the posterior of previour trial, so technically the prior of this trial
        else:
            negloglikelihood += np.log(agent.softmax_probs[1])
        agent.update_dynamic(best_arm, participant_rewards[t])
        #print(agent.softmax_probs)


    negloglikelihood = (-negloglikelihood)
    print(parameters, negloglikelihood)
    return negloglikelihood

# initial eta and decay setting
initial_parameters = [0.7, 0.3] # 0.3 #

# Define bounds for each parameter
#ucb_bounds = (0, 2.5758)  # Example bounds for eta parameter
decay_bounds = (0, 1)  # Example bounds for decay parameter
eta_bounds = (0, 1)

# Create a Bounds object
parameter_bounds = Bounds([eta_bounds[0], decay_bounds[0]], [eta_bounds[1],decay_bounds[1]])

# Iterate through participants and fit parameters separately
for participant in range(44):
    participant_choices = np_arraychoice_exp2[np_arraysubject_exp2 == (participant+1)]
    participant_rewards = np_arrayreward_exp2[np_arraysubject_exp2 == (participant+1)]
    print(participant)


    result = minimize(
        neg_fe_log_lik,
        initial_parameters, method = 'Powell', options={'xtol':1e-3, 'ftol': 1e-3},
        args=(participant_choices, participant_rewards),
        bounds=parameter_bounds
    )

        # Append results to the DataFrame
    results_df = results_df.append({
        'Participant': participant+1,
        'eta': result.x[0],
        'decay': result.x[1],
        'Optimal Likelihood': result.fun
    }, ignore_index=True)

results_df.to_csv('optimization_ai.csv', index=False)


"""    print("Optimization Result for Participant", participant + 1)
    print("Optimal Parameters:", result.x)
    print("Optimal Value of the Objective Function:", result.fun)"""



0
[0.7 0.3] 159.7436944608474
[0.38196601 0.3       ] 135.90701752708796
[0.61803399 0.3       ] 157.77435409215127
[0.23606798 0.3       ] 130.88537694560463
[0.19609637 0.3       ] 139.47992351787457
[0.29619319 0.3       ] 142.5563026409358
[0.25903376 0.3       ] 145.33602983206893
[0.22080018 0.3       ] 132.51535189918368
[0.24484013 0.3       ] 141.99799947886052
[0.2302362 0.3      ] 144.65505870965688
[0.23941864 0.3       ] 145.29134947171255
[0.23384044 0.3       ] 132.8031871336631
[0.23734782 0.3       ] 140.8265480798209
[0.23521713 0.3       ] 143.5599995983749
[0.23655683 0.3       ] 147.96381785210167
[0.23573464 0.3       ] 136.56367640190444
[0.23606798 0.38196601] 158.04226827974426
[0.23606798 0.61803399] 145.67120118804323
[0.23606798 0.76393202] 133.32551618954523
[0.23606798 0.85410197] 142.83918374131255
[0.23606798 0.74351569] 136.72020264576395
[0.23606798 0.78755164] 148.76474652372835
[0.23606798 0.77295392] 150.63919756074748
[0.23606798 0.75613368] 133.61

<ipython-input-9-dea678d29946>:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


[0.38196601 0.3       ] 156.64176667980533
[0.61803399 0.3       ] 147.3254945942666
[0.76393202 0.3       ] 174.214605032158
[0.53368275 0.3       ] 127.61686650681385
[0.51096137 0.3       ] 147.47863284726375
[0.5645668 0.3      ] 172.89772056333783
[0.54547941 0.3       ] 133.2404230371166
[0.52500395 0.3       ] 165.6406960487792
[0.53818867 0.3       ] 156.36629442761082
[0.53036774 0.3       ] 168.86388556626326
[0.53540386 0.3       ] 143.99443703239695
[0.53241653 0.3       ] 146.46705824196826
[0.53434015 0.3       ] 150.6415997553162
[0.5331991 0.3      ] 136.26042263494605
[0.53368275 0.38196601] 151.67612148037355
[0.53368275 0.61803399] 171.66079880345129
[0.53368275 0.23606798] 166.2125470282423
[0.53368275 0.4122693 ] 144.26220590014262
[0.53368275 0.49086442] 163.24708611416662
[0.53368275 0.42451607] 145.67715496960955
[0.53368275 0.41156846] 158.4656946098935
[0.53368275 0.41260263] 136.85567528047176
[0.53368275 0.41715316] 148.7155832934742
[0.53368275 0.41434078] 

<ipython-input-9-dea678d29946>:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


[0.38196601 0.3       ] 157.43266663285502
[0.61803399 0.3       ] 142.53317011774996
[0.76393202 0.3       ] 142.7136244507121
[0.6873123 0.3      ] 162.12120642381993
[0.52786405 0.3       ] 141.15141961249844
[0.47213595 0.3       ] 159.02017964002155
[0.5623059 0.3      ] 146.93569210884402
[0.50657781 0.3       ] 134.070129295127
[0.49342219 0.3       ] 139.89631116321303
[0.50983393 0.3       ] 138.17694433428335
[0.50213253 0.3       ] 150.8423428049551
[0.50487986 0.3       ] 133.8229550567383
[0.50540564 0.3       ] 142.71743728242788
[0.50383048 0.3       ] 140.6495594697992
[0.50447903 0.3       ] 157.60883011692584
[0.50487986 0.38196601] 159.62509152208904
[0.50487986 0.61803399] 150.15448006523195
[0.50487986 0.76393202] 161.8452658521772
[0.50487986 0.56371737] 155.48196577771506
[0.50487986 0.67376208] 141.6823617966639
[0.50487986 0.70820393] 144.03843269152173
[0.50487986 0.67699175] 149.16472015936586
[0.50487986 0.65247584] 155.8465457731536
[0.50487986 0.66563146] 

...

[0.38196601 0.3       ] 145.12862491376166
[0.61803399 0.3       ] 192.93805617810315
[0.23606798 0.3       ] 151.25032235421187
[0.34179397 0.3       ] 137.22368395797858
[0.31830786 0.3       ] 160.84872929305436
[0.35713833 0.3       ] 151.93483551422784
[0.33282308 0.3       ] 165.5241012902655
[0.347655 0.3     ] 143.47210729322575
[0.3383674 0.3      ] 156.88056061373692
[0.34403269 0.3       ] 142.75895719466857
[0.34048514 0.3       ] 141.48882626974438
[0.3421481 0.3      ] 167.43792641552946
[0.34146064 0.3       ] 178.27413642261374
[0.34179397 0.38196601] 150.79440286265566
[0.34179397 0.61803399] 152.07058351413858
[0.34179397 0.23606798] 139.47390154437076
[0.34179397 0.14589803] 178.74109022873594
[0.34179397 0.29179607] 176.46122138008505
[0.34179397 0.20162612] 153.66467227152773
[0.34179397 0.25735421] 149.36018954497885
[0.34179397 0.22291236] 142.62796784353444
[0.34179397 0.23535321] 157.4803234318075
[0.34179397 0.2441986 ] 151.3286013812704
[0.34179397 0.2391736 

<ipython-input-9-dea678d29946>:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


[0.38196601 0.3       ] 155.73706581733882
[0.61803399 0.3       ] 179.42374137348392
[0.23606798 0.3       ] 139.20201980542578
[0.14589803 0.3       ] 156.0364982532296
[0.26443103 0.3       ] 158.98150187179684
[0.23573464 0.3       ] 145.1261250389953
[0.2469017 0.3      ] 130.78117468598114
[0.24174021 0.3       ] 142.77730152379837
[0.25359731 0.3       ] 153.24422883248246
[0.24945919 0.3       ] 143.114699370943
[0.24493019 0.3       ] 135.54613856253832
[0.24787857 0.3       ] 163.12450688890155
[0.24614865 0.3       ] 181.27588242381515
[0.24727483 0.3       ] 139.97739361049906
[0.24656836 0.3       ] 161.93846681949745
[0.2469017  0.38196601] 164.9627731345317
[0.2469017  0.61803399] 132.2237646450744
[0.2469017  0.76393202] 145.8764020387469
[0.2469017  0.61403717] 155.54273495194025
[0.2469017  0.68979991] 152.0065778901755
[0.2469017  0.64544613] 142.22670166941197
[0.2469017 0.6285045] 166.62715298261304
[0.2469017  0.62203337] 153.71273890819026
[0.2469017  0.61956162]

<ipython-input-9-dea678d29946>:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


'    print("Optimization Result for Participant", participant + 1)\n    print("Optimal Parameters:", result.x)\n    print("Optimal Value of the Objective Function:", result.fun)'

In [ ]:
print(result)

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 262.3486128551278
       x: [ 1.246e-01]
     nit: 3
   direc: [[ 1.000e+00]]
    nfev: 42
